In [1]:
#Dependencies 
import requests
import json
import pandas as pd
import numpy as np
import os
import csv

#from config_edu import api_key

crimefile_path = "datasource/crime_data.csv"
crimefile_df = pd.read_csv(crimefile_path)
crimefile_df.head(1)

,jurisdiction,includes_jails,year,prisoner_count,crime_reporting_change,crimes_estimated,state_population,violent_crime_total,murder_manslaughter,rape_legacy,rape_revised,robbery,agg_assault,property_crime_total,burglary,larceny,vehicle_theft
0,FEDERAL,False,2001,149852,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [2]:
# Removing Federal Data as there are lot of NaN data.
crimefile_df = crimefile_df.loc[crimefile_df["jurisdiction"]!= "FEDERAL",:]
crimefile_df.head(1)

,jurisdiction,includes_jails,year,prisoner_count,crime_reporting_change,crimes_estimated,state_population,violent_crime_total,murder_manslaughter,rape_legacy,rape_revised,robbery,agg_assault,property_crime_total,burglary,larceny,vehicle_theft
1,ALABAMA,False,2001,24741,False,False,4468912.0,19582.0,379.0,1369.0,NaN,5584.0,12250.0,173253.0,40642.0,119992.0,12619.0


In [3]:
# Take one state
crimefile_df = crimefile_df.loc[crimefile_df["jurisdiction"]=="ARIZONA",:]
crimefile_df.head(1)

,jurisdiction,includes_jails,year,prisoner_count,crime_reporting_change,crimes_estimated,state_population,violent_crime_total,murder_manslaughter,rape_legacy,rape_revised,robbery,agg_assault,property_crime_total,burglary,larceny,vehicle_theft
3,ARIZONA,False,2001,27710,False,False,5306966.0,28675.0,400.0,1518.0,NaN,8868.0,17889.0,293874.0,54821.0,186850.0,52203.0


In [4]:
#Read Election Data 2000
election2000 = "datasource/Election Data 2000.csv"
election2000_df = pd.read_csv (election2000)
election2000_df.head(1)

,State,EV,EV.1,Total Vote,G,B,N,Margin,%Margin,Gore,Bush,Nader,Other,Gore.1,Bush.1,Nader.1,Other.1
0,Alabama,9,0,"1,672,551",2,1,3,"248,807",14.88%,41.59%,56.47%,1.10%,0.85%,"695,602","944,409","18,349","14,191"


In [5]:
#Add Column for year
election2000_df ["Year"] = 2000 
election2000_df.head(1)

,State,EV,EV.1,Total Vote,G,B,N,Margin,%Margin,Gore,Bush,Nader,Other,Gore.1,Bush.1,Nader.1,Other.1,Year
0,Alabama,9,0,"1,672,551",2,1,3,"248,807",14.88%,41.59%,56.47%,1.10%,0.85%,"695,602","944,409","18,349","14,191",2000


In [6]:
#identify columns & Rename Total Vote 
columns = election2000_df.columns 
election2000_df.rename(columns ={'Total\xa0Vote':'Total Vote'}, inplace = True)
election2000_df.head(1)

,State,EV,EV.1,Total Vote,G,B,N,Margin,%Margin,Gore,Bush,Nader,Other,Gore.1,Bush.1,Nader.1,Other.1,Year
0,Alabama,9,0,"1,672,551",2,1,3,"248,807",14.88%,41.59%,56.47%,1.10%,0.85%,"695,602","944,409","18,349","14,191",2000


In [7]:
#Change Columns from String to Floats
election2000_df['Other.1'] = election2000_df['Other.1'].str.replace(',','').astype(float)
election2000_df['Nader.1'] = election2000_df['Nader.1'].str.replace(',','').astype(float)
election2000_df['Bush.1'] = election2000_df['Bush.1'].str.replace(',','').astype(float)
election2000_df['Gore.1'] = election2000_df['Gore.1'].str.replace(',','').astype(float)
election2000_df['Total Vote'] = election2000_df['Total Vote'].str.replace(',','').astype(float)
election2000_df.head(1)



,State,EV,EV.1,Total Vote,G,B,N,Margin,%Margin,Gore,Bush,Nader,Other,Gore.1,Bush.1,Nader.1,Other.1,Year
0,Alabama,9,0,1672551.0,2,1,3,"248,807",14.88%,41.59%,56.47%,1.10%,0.85%,695602.0,944409.0,18349.0,14191.0,2000


In [8]:
#Create and Add Other Party - every Party outside Republican and Democrat - Total votes outside of Rep & Dem
election2000_df["Other Party"] = election2000_df['Nader.1'] + election2000_df['Other.1']
election2000_df.head(1)

,State,EV,EV.1,Total Vote,G,B,N,Margin,%Margin,Gore,Bush,Nader,Other,Gore.1,Bush.1,Nader.1,Other.1,Year,Other Party
0,Alabama,9,0,1672551.0,2,1,3,"248,807",14.88%,41.59%,56.47%,1.10%,0.85%,695602.0,944409.0,18349.0,14191.0,2000,32540.0


In [9]:
#Rename Presidents to their Parties
election2000_df.rename(columns = {'Gore.1': 'Democrat',
                                  'Bush.1': 'Republican',
                                  'Gore': 'Democrat_Perc',
                                  'Bush': 'Republican_Perc'}, inplace=True)

election2000_df.head(1)


,State,EV,EV.1,Total Vote,G,B,N,Margin,%Margin,Democrat_Perc,Republican_Perc,Nader,Other,Democrat,Republican,Nader.1,Other.1,Year,Other Party
0,Alabama,9,0,1672551.0,2,1,3,"248,807",14.88%,41.59%,56.47%,1.10%,0.85%,695602.0,944409.0,18349.0,14191.0,2000,32540.0


In [10]:
#Read 2004 Election Data, create Year column,create Other Party column, Replace all str with floats
election2004 = "datasource/Election Data 2004.csv"
election2004_df = pd.read_csv (election2004)
election2004_df ["Year"] = 2004
election2004_df.head(1)

,State,EV,EV.1,Total Vote,B,K,Margin,%Margin,Bush,Kerry,Other,Bush.1,Kerry.1,Other.1,Year
0,Alabama,9,0,"1,883,449",1,2,"482,461",25.62%,62.46%,36.84%,0.70%,"1,176,394","693,933","13,122",2004


In [11]:
#identify columns & Rename Total Vote 
columns = election2004_df.columns 
election2004_df.rename(columns ={'Total\xa0Vote':'Total Vote'}, inplace = True)
election2004_df.head(1)

,State,EV,EV.1,Total Vote,B,K,Margin,%Margin,Bush,Kerry,Other,Bush.1,Kerry.1,Other.1,Year
0,Alabama,9,0,"1,883,449",1,2,"482,461",25.62%,62.46%,36.84%,0.70%,"1,176,394","693,933","13,122",2004


In [27]:
#Change Columns from String to Floats

election2004_df['Kerry.1'] = election2004_df['Kerry.1'].replace(',','').astype(float)
election2004_df['Bush.1'] = election2004_df['Bush.1'].replace(',','').astype(float)
election2004_df['Total Vote'] = election2004_df['Total Vote'].replace(',','').astype(float)
election2004_df.head(1)

,State,EV,EV.1,Total Vote,B,K,Margin,%Margin,Bush,Kerry,Other,Bush.1,Kerry.1,Other.1,Year
0,Alabama,9,0,1883449.0,1,2,"482,461",25.62%,62.46%,36.84%,0.70%,1176394.0,693933.0,13122.0,2004


In [28]:
#Rename Presidents to their Parties
election2004_df.rename(columns = {'Kerry.1': 'Democrat',
                                  'Bush.1': 'Republican',
                                  'Kerry': 'Democrat_Perc',
                                  'Bush': 'Republican_Perc',
                                  'Other.1': "Other Party"}, inplace=True)
election2004_df.head(1)

,State,EV,EV.1,Total Vote,B,K,Margin,%Margin,Republican_Perc,Democrat_Perc,Other,Republican,Democrat,Other Party,Year
0,Alabama,9,0,1883449.0,1,2,"482,461",25.62%,62.46%,36.84%,0.70%,1176394.0,693933.0,13122.0,2004


In [29]:
#Read Election Data 2008,create Year column, 
#rename Total Vote, create Other Party column, Replace all str with floats
election2008 = "datasource/Election Data 2008.csv"
election2008_df = pd.read_csv (election2008)
election2008_df ["Year"] = 2008
election2008_df.head(1)

,State,EV,EV.1,Total Vote,O,M,Margin,%Margin,Obama,McCain,Other,Obama.1,McCain.1,Other.1,Year
0,Alabama,0,9,"2,099,819",2,1,"453,067",21.58%,38.74%,60.32%,0.94%,"813,479","1,266,546","19,794",2008


In [30]:
#List Columns
columns = election2008_df.columns
columns

Index(['State', 'EV', 'EV.1', 'Total Vote', 'O', 'M', 'Margin', '%Margin',
       'Obama', 'McCain', 'Other', 'Obama.1', 'McCain.1', 'Other.1', 'Year'],
      dtype='object')

In [31]:
#Find name to column "Total Vote"
columns[3]

'Total\xa0Vote'

In [33]:
#Rename Total Vote Column
columns = election2008_df.columns 
election2008_df.rename(columns ={'Total\xa0Vote':'Total Vote'}, inplace = True)
election2008_df.head(1)

,State,EV,EV.1,Total Vote,O,M,Margin,%Margin,Obama,McCain,Other,Obama.1,McCain.1,Other.1,Year
0,Alabama,0,9,"2,099,819",2,1,"453,067",21.58%,38.74%,60.32%,0.94%,"813,479","1,266,546","19,794",2008


In [35]:
#Change Columns from String to Floats
election2008_df['Other.1'] = election2008_df['Other.1'].str.replace(',','').astype(float)
election2008_df['Obama.1'] = election2008_df['Obama.1'].str.replace(',','').astype(float)
election2008_df['McCain.1'] = election2008_df['McCain.1'].str.replace(',','').astype(float)
election2008_df['Total Vote'] = election2008_df['Total Vote'].str.replace(',','').astype(float)
election2008_df.head(1)

,State,EV,EV.1,Total Vote,O,M,Margin,%Margin,Obama,McCain,Other,Obama.1,McCain.1,Other.1,Year
0,Alabama,0,9,2099819.0,2,1,"453,067",21.58%,38.74%,60.32%,0.94%,813479.0,1266546.0,19794.0,2008


In [36]:
#Rename Presidents to their Parties
election2008_df.rename(columns = {'Obama.1': 'Democrat',
                                  'McCain.1': 'Republican',
                                  'Obama': 'Democrat_Perc',
                                  'McCain': 'Republican_Perc',
                                  'Other.1': 'Other Party'}, inplace=True)

election2008_df.head(1)

,State,EV,EV.1,Total Vote,O,M,Margin,%Margin,Democrat_Perc,Republican_Perc,Other,Democrat,Republican,Other Party,Year
0,Alabama,0,9,2099819.0,2,1,"453,067",21.58%,38.74%,60.32%,0.94%,813479.0,1266546.0,19794.0,2008


In [37]:
#Read Election Data 2012
election2012 = "datasource/Election Data 2012.csv"
election2012_df = pd.read_csv (election2012)
election2012_df ["Year"] = 2012
election2012_df.head(1)

,State,EV,EV.1,Total Vote,O,R,Margin,%Margin,Obama,Romney,Other,Obama.1,Romney.1,Other.1,Year
0,Alabama,0.0,9.0,"2,074,338",2.0,1.0,"460,229",22.19%,38.36%,60.55%,1.10%,"795,696","1,255,925","22,717",2012


In [38]:
#List Columns
columns = election2012_df.columns
columns

Index(['State', 'EV', 'EV.1', 'Total Vote', 'O', 'R', 'Margin', '%Margin',
       'Obama', 'Romney', 'Other', 'Obama.1', 'Romney.1', 'Other.1', 'Year'],
      dtype='object')

In [39]:
#Find name to column "Total Vote"
columns[3]

'Total\xa0Vote'

In [40]:
#Rename Total Vote Column
columns = election2012_df.columns 
election2012_df.rename(columns ={'Total\xa0Vote':'Total Vote'}, inplace = True)
election2012_df.head(1)

,State,EV,EV.1,Total Vote,O,R,Margin,%Margin,Obama,Romney,Other,Obama.1,Romney.1,Other.1,Year
0,Alabama,0.0,9.0,"2,074,338",2.0,1.0,"460,229",22.19%,38.36%,60.55%,1.10%,"795,696","1,255,925","22,717",2012


In [44]:
#Change Columns from String to Floats
election2012_df['Other.1'] = election2012_df['Other.1'].replace(',','').astype(float)
election2012_df['Obama.1'] = election2012_df['Obama.1'].str.replace(',','').astype(float)
election2012_df['Romney.1'] = election2012_df['Romney.1'].str.replace(',','').astype(float)
election2012_df['Total Vote'] = election2012_df['Total Vote'].str.replace(',','').astype(float)
election2012_df.head(1)

,State,EV,EV.1,Total Vote,O,R,Margin,%Margin,Obama,Romney,Other,Obama.1,Romney.1,Other.1,Year
0,Alabama,0.0,9.0,2074338.0,2.0,1.0,"460,229",22.19%,38.36%,60.55%,1.10%,795696.0,1255925.0,22717.0,2012


In [45]:
#Rename Presidents to their Parties
election2012_df.rename(columns = {'Obama.1': 'Democrat',
                                  'Romney.1': 'Republican',
                                  'Obama': 'Democrat_Perc',
                                  'Romney': 'Republican_Perc',}, inplace=True)
election2012_df ["Other Party"] = election2012_df['Other.1']
election2012_df.head(1)

,State,EV,EV.1,Total Vote,O,R,Margin,%Margin,Democrat_Perc,Republican_Perc,Other,Democrat,Republican,Other.1,Year,Other Party
0,Alabama,0.0,9.0,2074338.0,2.0,1.0,"460,229",22.19%,38.36%,60.55%,1.10%,795696.0,1255925.0,22717.0,2012,22717.0


In [46]:
#Read Election Data 2016
election2016 = "datasource/Election Data 2016.csv"
election2016_df = pd.read_csv (election2016)
election2016_df ["Year"] = 2016
election2016_df.head(1)

,State,EV,EV.1,EV.2,Total Vote,C,T,J,Margin,%Margin,Clinton,Trump,Johnson,Other,Clinton.1,Trump.1,Johnson.1,Other.1,Year
0,Texas,36,0,2,"8,993,166",2,1,3,"807,179",8.98%,43.12%,52.10%,3.15%,1.63%,"3,877,868","4,685,047","283,492","146,759",2016


In [47]:
#List Columns
columns = election2016_df.columns
columns

Index(['State', 'EV', 'EV.1', 'EV.2', 'Total Vote', 'C', 'T', 'J', 'Margin',
       '%Margin', 'Clinton', 'Trump', 'Johnson', 'Other', 'Clinton.1',
       'Trump.1', 'Johnson.1', 'Other.1', 'Year'],
      dtype='object')

In [48]:
#Find name to column "Total Vote"
columns[4]

'Total\xa0Vote'

In [49]:
#Rename Total Vote Column
columns = election2016_df.columns 
election2016_df.rename(columns ={'Total\xa0Vote':'Total Vote'}, inplace = True)
election2016_df.head(1)

,State,EV,EV.1,EV.2,Total Vote,C,T,J,Margin,%Margin,Clinton,Trump,Johnson,Other,Clinton.1,Trump.1,Johnson.1,Other.1,Year
0,Texas,36,0,2,"8,993,166",2,1,3,"807,179",8.98%,43.12%,52.10%,3.15%,1.63%,"3,877,868","4,685,047","283,492","146,759",2016


In [50]:
#Change Columns from String to Floats
election2016_df['Other.1'] = election2016_df['Other.1'].str.replace(',','').astype(float)
election2016_df['Johnson.1'] = election2016_df['Johnson.1'].str.replace(',','').astype(float)
election2016_df['Clinton.1'] = election2016_df['Clinton.1'].str.replace(',','').astype(float)
election2016_df['Trump.1'] = election2016_df['Trump.1'].str.replace(',','').astype(float)
election2016_df['Total Vote'] = election2016_df['Total Vote'].str.replace(',','').astype(float)
election2016_df.head(1)

,State,EV,EV.1,EV.2,Total Vote,C,T,J,Margin,%Margin,Clinton,Trump,Johnson,Other,Clinton.1,Trump.1,Johnson.1,Other.1,Year
0,Texas,36,0,2,8993166.0,2,1,3,"807,179",8.98%,43.12%,52.10%,3.15%,1.63%,3877868.0,4685047.0,283492.0,146759.0,2016


In [51]:
#Rename Presidents to their Parties
election2016_df.rename(columns = {'Clinton.1': 'Democrat',
                                  'Trump.1': 'Republican',
                                  'Clinton': 'Democrat_Perc',
                                  'Trump': 'Republican_Perc',}, inplace=True)
election2016_df ["Other Party"] = election2016_df['Other.1']+ election2016_df['Other.1']
election2016_df.head(1)

,State,EV,EV.1,EV.2,Total Vote,C,T,J,Margin,%Margin,Democrat_Perc,Republican_Perc,Johnson,Other,Democrat,Republican,Johnson.1,Other.1,Year,Other Party
0,Texas,36,0,2,8993166.0,2,1,3,"807,179",8.98%,43.12%,52.10%,3.15%,1.63%,3877868.0,4685047.0,283492.0,146759.0,2016,293518.0


In [ ]:
#Democrat, Republican, Other Party - Democrat by state/ by Total crime by year

crime_election2000 = pd.Dataframe({crimedata_df['violent_crime_total'], election2000_df['Democrat']})

In [ ]:
#Maine
iloc[0,:]
Maine_df = election2000_df.iloc[19,:] + crimefile_df['violent_crime_total']
Maine_df.head(1)

In [ ]:
election_crimedata = election2000_df.append(crimefile_df, sort=False)

In [53]:
crimefile_df = pd.DataFrame(crimefile_df, columns = ['jurisdiction','year','violent_crime_total','state_population',
                                                     'murder_manslaughter', 'rape_legacy','rape_revised', 'robbery', 
                                                     'agg_assault', 'property_crime_total','burglary', 'larceny', 'vehicle_theft'])
crimefile_df.head()

,jurisdiction,year,violent_crime_total,state_population,murder_manslaughter,rape_legacy,rape_revised,robbery,agg_assault,property_crime_total,burglary,larceny,vehicle_theft
3,ARIZONA,2001,28675.0,5306966.0,400.0,1518.0,NaN,8868.0,17889.0,293874.0,54821.0,186850.0,52203.0
54,ARIZONA,2002,30171.0,5441125.0,387.0,1608.0,NaN,8000.0,20176.0,318296.0,59087.0,201541.0,57668.0
105,ARIZONA,2003,28638.0,5579222.0,441.0,1856.0,NaN,7619.0,18722.0,314335.0,58613.0,198725.0,56997.0
156,ARIZONA,2004,28952.0,5739879.0,414.0,1896.0,NaN,7721.0,18921.0,291203.0,56885.0,179012.0,55306.0
207,ARIZONA,2005,30478.0,5953007.0,445.0,2006.0,NaN,8579.0,19448.0,287345.0,56328.0,176112.0,54905.0


In [58]:
electiondata = pd.DataFrame(election2000_df, columns =['State','Year', 'Democrat', 'Republican', 'Other Party' ], copy=True)
electiondata.head(1)

,State,Year,Democrat,Republican,Other Party
0,Alabama,2000,695602.0,944409.0,32540.0


In [60]:
election2000_df.index('Maine')

TypeError: 'RangeIndex' object is not callable

In [64]:
crime_election_combined = pd.concat([electiondata, crimefile_df], axis=1, join_axes=[electiondata.index])
crime_election_combined.head(10)

,State,Year,Democrat,Republican,Other Party,jurisdiction,year,violent_crime_total,state_population,murder_manslaughter,rape_legacy,rape_revised,robbery,agg_assault,property_crime_total,burglary,larceny,vehicle_theft
0,Alabama,2000,695602.0,944409.0,32540.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Alaska,2000,79004.0,167398.0,39158.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Arizona,2000,685341.0,781652.0,67120.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Arkansas,2000,422768.0,472940.0,26073.0,ARIZONA,2001.0,28675.0,5306966.0,400.0,1518.0,NaN,8868.0,17889.0,293874.0,54821.0,186850.0,52203.0
4,California,2000,5861203.0,4567429.0,537224.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Colorado,2000,738227.0,883745.0,119393.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,Connecticut,2000,816015.0,561094.0,82416.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,Delaware,2000,180068.0,137288.0,10266.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,D. C.,2000,171923.0,18073.0,11898.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,Florida,2000,2912253.0,2912790.0,138067.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
